# 1. Logistic Regression

In [39]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
#from config import db_password
from sqlalchemy import inspect
import matplotlib.pyplot as plt
import pandas as pd

business_df = pd.read_csv("../../Data/business_census_merged_dataset.csv")

# Categorizing restaurants based on stars ratings
business_df["Category"] = pd.cut(business_df["Stars_Rating"],bins=[0.9,2,3,4,5],
                                 labels=["Poor","Average","Good","Successful"])

def changeStatus(status):
    if status == "Poor":
        return 0
    elif status == "Average":
        return 1
    elif status ==  "Good":
        return 2
    else:
        return 3

business_df['Category_Encoded'] = business_df["Category"].apply(changeStatus)
business_df["Category_Encoded"] = pd.to_numeric(business_df["Category_Encoded"])


business_df["Median_Income(dollars)"] = pd.to_numeric(business_df["Median_Income(dollars)"], errors='coerce')
business_df["Mean_Income(dollars)"] = pd.to_numeric(business_df["Mean_Income(dollars)"], errors='coerce')

business_df = business_df.dropna(subset=['Median_Income(dollars)'])

In [57]:
business_df

,Restaurant_ID,Restaurants_Name,Address,City,State,Postal_Code,Latitude,Longitude,Stars_Rating,Review_Count,...,Happy_Hour,Noise_Level,Dietary_Restrictions,Total_Estimate_Households_per_Zip,Total_Estimate_Married-couple_Family_households,Total_Estimate_Nonfamily_households,Median_Income(dollars),Mean_Income(dollars),Category,Category_Encoded
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,0,1,0,7823,1743,5771,60676,91522,Good,2
1,WYYdQDjx-DsCanlP0DpImQ,J Sushi,243 S 10th St,Philadelphia,PA,19107,39.947033,-75.157434,4.5,72,...,0,4,0,7823,1743,5771,60676,91522,Successful,3
2,ngvE1G9IckUO-ljZBFo23w,Mumbai Bistro,930 Locust St,Philadelphia,PA,19107,39.947207,-75.157252,4.0,217,...,0,1,0,7823,1743,5771,60676,91522,Good,2
3,IgNDlyLERBwFGphgcC5VDQ,Cosi,1128 Walnut St,Philadelphia,PA,19107,39.948751,-75.160449,2.5,63,...,0,4,0,7823,1743,5771,60676,91522,Average,1
4,L_sXNadtVHjxMw7Yhvkj9Q,Naoki Ramen,915 Race St,Philadelphia,PA,19107,39.955454,-75.154900,4.0,92,...,0,2,0,7823,1743,5771,60676,91522,Good,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24734,IFUgn1pFknZQ9OpN9EmyYw,Hudock's Frozen Custard Stand,544 Salem Quinton Rd,Salem,NJ,8079,39.560427,-75.448576,4.0,35,...,0,2,0,4018,1397,1574,42077,67275,Good,2
24735,tZy556-b1uO9pcy0WsumIA,Charlie's Restaurant & Catering,6129 E Saint Bernard Hwy,Violet,LA,70092,29.900428,-89.901686,4.0,71,...,0,2,0,2690,1417,624,44753,62162,Good,2
24736,jMStvE-tQzSpRCAO0nAE6g,Red Robin Gourmet Burgers and Brews,4605 S Landing Way,Tucson,AZ,32246,32.167311,-110.986422,3.0,159,...,1,4,0,21387,9033,8741,61730,76508,Average,1
24737,r4oV3EWnJX-1hjhNcKMI7w,Ortino's Northside,1355 Gravel Pike,Perkiomenville,PA,18074,40.291583,-75.481124,3.5,32,...,1,4,0,2298,1512,495,99333,132779,Good,2


In [58]:
business_df["Category"].value_counts()

Good          14021
Successful     4884
Average        4820
Poor           1014
Name: Category, dtype: int64

In [59]:
business_df.columns

Index(['Restaurant_ID', 'Restaurants_Name', 'Address', 'City', 'State',
       'Postal_Code', 'Latitude', 'Longitude', 'Stars_Rating', 'Review_Count',
       'Restaurants_Delivery', 'Outdoor_Seating', 'Accepts_CreditCards',
       'Price_Range', 'Alcohol', 'Good_For_Kids', 'Reservations',
       'Restaurants_TakeOut', 'WiFi', 'Good_For_Groups',
       'Wheelchair_Accessible', 'Happy_Hour', 'Noise_Level',
       'Dietary_Restrictions', 'Total_Estimate_Households_per_Zip',
       'Total_Estimate_Married-couple_Family_households',
       'Total_Estimate_Nonfamily_households', 'Median_Income(dollars)',
       'Mean_Income(dollars)', 'Category', 'Category_Encoded'],
      dtype='object')

In [60]:
# Define features set
X = business_df[['Review_Count', 'Restaurants_Delivery', 'Outdoor_Seating',
       'Accepts_CreditCards', 'Price_Range', 'Alcohol', 'Good_For_Kids',
       'Reservations', 'Restaurants_TakeOut', 'WiFi', 'Good_For_Groups',
       'Wheelchair_Accessible', 'Happy_Hour', 'Noise_Level',
       'Dietary_Restrictions',
                'Total_Estimate_Married-couple_Family_households',
       'Total_Estimate_Nonfamily_households',
                'Median_Income(dollars)', 
                 'Total_Estimate_Households_per_Zip']]

In [61]:
# Define the target
y = business_df["Category_Encoded"]
y

0        2
1        3
2        2
3        1
4        2
        ..
24734    2
24735    2
24736    1
24737    2
24738    1
Name: Category_Encoded, Length: 24739, dtype: int64

In [62]:
# Split the model into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y, 
                                                   random_state=1, 
                                                    stratify=y)

In [63]:
# Instantiate a Logistic Regression Model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="lbfgs", random_state=1)
model

LogisticRegression(random_state=1)

In [64]:
# Train the model
model.fit(X_train, y_train)

/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [65]:
# Make predictions
y_pred = model.predict(X_test)

In [66]:
# Testing Accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5666936135812449

In [67]:
# Training balanced accuracy
y_pred_train = model.predict(X_train)
accuracy_score(y_train, y_pred_train)

0.5667780532499731

In [68]:
# Calculate the confusion matrix 
from sklearn.metrics import confusion_matrix, classification_report

matrix=confusion_matrix(y_test, y_pred)
print(matrix)

[[   0    0  254    0]
 [   0    0 1205    0]
 [   0    0 3505    0]
 [   0    0 1221    0]]


In [69]:
report=classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       254
           1       0.00      0.00      0.00      1205
           2       0.57      1.00      0.72      3505
           3       0.00      0.00      0.00      1221

    accuracy                           0.57      6185
   macro avg       0.14      0.25      0.18      6185
weighted avg       0.32      0.57      0.41      6185



/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

# Logistic Regression with Oversampling

In [70]:
from collections import Counter
Counter(y)

Counter({2: 14021, 3: 4884, 1: 4820, 0: 1014})

It seems like the classes are unbalanced, which might bias the results toward the majority classes. 

# Naive Random Oversampling


In random oversampling, instances of the minority class are randomly selected and added to the training set until the majority and minority classes are balanced.  

In [71]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({3: 10516, 1: 10516, 2: 10516, 0: 10516})

In [72]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [73]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.35061230492347084

In [74]:
y_pred_train = model.predict(X_train)
balanced_accuracy_score(y_train, y_pred_train)

0.33944144814084076

In [77]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[ 142,   66,   35,   11],
       [ 449,  320,  356,   80],
       [ 857,  705, 1411,  532],
       [ 342,  262,  403,  214]])

In [78]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.08      0.56      0.72      0.14      0.64      0.40       254
          1       0.24      0.27      0.79      0.25      0.46      0.20      1205
          2       0.64      0.40      0.70      0.49      0.53      0.27      3505
          3       0.26      0.18      0.87      0.21      0.39      0.14      1221

avg / total       0.46      0.34      0.76      0.38      0.49      0.24      6185



# SMOTE Oversampling

In SMOTE, like naive random oversampling, the size of the minority is increased. The key difference with random oversampling is how the minority class is increased in size. 

However, in SMOTE, new instances are interpolated. That is, for an instance from the minority class, new values are generated based on its distance from its neighbors.

**Random oversampling draws from existing observations, whereas SMOTE generates synthetic observations**.

In [79]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy="auto").fit_resample(X_train, y_train)

from collections import Counter

Counter(y_resampled)

Counter({3: 10516, 1: 10516, 2: 10516, 0: 10516})

In [80]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [82]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.32592720218625193

In [83]:
# Training balanced accuracy
y_pred_train = model.predict(X_train)
accuracy_score(y_train, y_pred_train)

0.2944378570658618

In [84]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 137,   80,   20,   17],
       [ 565,  326,  215,   99],
       [1050,  829, 1171,  455],
       [ 334,  319,  373,  195]])

In [86]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.07      0.54      0.67      0.12      0.60      0.36       254
          1       0.21      0.27      0.75      0.24      0.45      0.19      1205
          2       0.66      0.33      0.77      0.44      0.51      0.25      3505
          3       0.25      0.16      0.88      0.20      0.38      0.13      1221

avg / total       0.47      0.30      0.79      0.34      0.47      0.22      6185



# Cluster Centroids Undersampling

Akin to SMOTE, the algorithm identifies clusters of the majority class, then generates synthetic data points, called centroids, that are representative of the clusters. The majority class is then undersampled down to the size of the minority class. 

In [87]:
# Resample the data using the ClusterCentroids resampler

from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

from collections import Counter

Counter(y_resampled)

Counter({0: 760, 1: 760, 2: 760, 3: 760})

In [92]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [93]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [94]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.3034147139091991

In [95]:
# Training balanced accuracy
y_pred_train = model.predict(X_train)
accuracy_score(y_train, y_pred_train)

0.21176026732779993

In [96]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 154,   63,   35,    2],
       [ 640,  301,  202,   62],
       [1557,  698,  616,  634],
       [ 554,  235,  210,  222]])

In [97]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.05      0.61      0.54      0.10      0.57      0.33       254
          1       0.23      0.25      0.80      0.24      0.45      0.19      1205
          2       0.58      0.18      0.83      0.27      0.38      0.14      3505
          3       0.24      0.18      0.86      0.21      0.40      0.15      1221

avg / total       0.42      0.21      0.82      0.24      0.41      0.16      6185



# Combination of Over and Under-Sampling

A downside of oversampling with SMOTE is its reliance on the immediate neighbors of a data point. Because the algorithm doesn't see the overall distribution of data, the new data it creates can be heavily influenced by outliers. 

A downside of undersampling is that it involves loss of data and is not an option when the dataset is small. 

SMOTEEN combines the SMOTE and Edited Nearest Neighbors (ENN) algorithms. It includes the following steps: 
1. Oversample the minority class with SMOTE.
2. Clean the resulting data with an undersampling strategy. If the two of the nearest neighbors of a data point belong to different classes, the data point is dropped.

In [102]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 6992, 1: 3106, 2: 1124, 3: 2741})

In [103]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [104]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.2663659911134231

In [105]:
# Training balanced accuracy
y_pred_train = model.predict(X_train)
accuracy_score(y_train, y_pred_train)

0.05928640724372103

In [106]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 246,    0,    0,    8],
       [1120,    3,    0,   82],
       [3128,    6,    1,  370],
       [1105,    1,    0,  115]])

In [107]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.04      0.97      0.10      0.08      0.31      0.10       254
          1       0.30      0.00      1.00      0.00      0.05      0.00      1205
          2       1.00      0.00      1.00      0.00      0.02      0.00      3505
          3       0.20      0.09      0.91      0.13      0.29      0.08      1221

avg / total       0.67      0.06      0.94      0.03      0.09      0.02      6185

